In [5]:
import json
import os
import numpy as np
from copy import deepcopy
import itertools

In [6]:

"""
Pf1 :w1w2,w1w5,w2w5,w1w3,w4w5,w2w4,w1w4,w3w4,w3w5,w2w3,w1,w4,w3,w2,w5 
Pf2 :w3w6,w3w5,w5w6,w2w5,w1w3,w2w6,w1w5,w1w2,w2w3,w1w6,w1,w2,w3,w5,w6 
Pf3 :w2w4,w1w2,w3w4,w2w3,w1w3,w1w4,w1,w2,w3,w4
Pw1 : f3, f1, f2
Pw2 : f2f3, f1f3, f1f2, f1, f2, f3 
Pw3 : f1, f2
Pw4 : f1, f3, f2
Pw5 : f2, f3
Pw6 :f1f3,f3,f1

μF= f1   f2   f3   ∅    μW= f1   f2   f3   ∅
    w1w2 w3w5 w2w4 w6       w3w4 w2w5 w1w2 w6

"""
P_f = [[[1,2],[1,5],[2,5],[1,3],[4,5],[2,4],[1,4],[3,4],[3,5],[2,3],[1],[4],[3],[5]],
        [[3,6],[3,5],[5,6],[2,5],[1,3],[2,6],[1,5],[1,2],[2,3],[1,6],[1],[2],[3],[5],[6]],
        [[2,4],[1,2],[3,4],[2,3],[1,3],[1,4],[1],[2],[3],[4]]]
P_w = [[[3],[1],[2]],
        [[2,3],[1,3],[1,2],[1],[2],[3]],
        [[1],[2]],
        [[1],[3],[2]],
        [[2],[3]],
        [[1,3],[3],[1],[2]]]

# matching, index are firm and values are worker. None existing values are unmatched workers, empty values are unmatched firms.
n_ws = 6
n_fs = 3
mu_f = [[1,2],[3,5],[2,4]]
mu_w = [[3,4],[2,5],[1,2]]

def print_preference(P_f,P_w):
        print(
                [f'Pf{i+1}:'+','.join(["".join([f"w{w}" for w in ws]) for ws in pfi]) for (i,pfi) in enumerate(P_f)],
        )
        print(
                [f'Pw{i+1}:'+','.join(["".join([f"f{f}" for f in fs]) for fs in pwi]) for (i,pwi) in enumerate(P_w)]
        )
print_preference(P_f,P_w)

['Pf1:w1w2,w1w5,w2w5,w1w3,w4w5,w2w4,w1w4,w3w4,w3w5,w2w3,w1,w4,w3,w5', 'Pf2:w3w6,w3w5,w5w6,w2w5,w1w3,w2w6,w1w5,w1w2,w2w3,w1w6,w1,w2,w3,w5,w6', 'Pf3:w2w4,w1w2,w3w4,w2w3,w1w3,w1w4,w1,w2,w3,w4']
['Pw1:f3,f1,f2', 'Pw2:f2f3,f1f3,f1f2,f1,f2,f3', 'Pw3:f1,f2', 'Pw4:f1,f3,f2', 'Pw5:f2,f3', 'Pw6:f1f3,f3,f1,f2']


In [7]:

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def index_from_match(query,matches):
    result = []
    for i,match in enumerate(matches):
        if query in match:
            result.append(i+1)
    print('ifm',query,matches,result)
    return result

def DA_algorithm(P_f,P_w):
    P_f = deepcopy(P_f)
    P_w = deepcopy(P_w)
    w_iters = max([len(pw) for pw in P_w])
    f_iters = max([len(pf) for pf in P_f])
    mu_f = []
    f = 0
    w = 0
    def DA_helper(f): # recursive defination. work on a reduced preference list.
        try:
            propose_match = P_f[f].pop(0)
            print("ini_prop:",propose_match)
        except:
            mu_f.append([]) # no item left to pop
            return
        for w_i in propose_match: #check if f is in P_w_i
            try:
                P_w[w_i-1].index([f+1])
            except:
                return DA_helper(f) # directly move to next preference in P_f
        for w_i in propose_match:
            conflict_fs = index_from_match(w_i,mu_f)
            result_fs = []
            # check if current f can be grouped other f for w_i
            for comb in reversed(list(powerset(sorted([f+1]+conflict_fs)))): # ordered to start from larger group
                try: 
                    g_prefer = P_w[w_i-1].index(list(comb)) # can be grouped
                    for i in comb:
                        try:
                            conflict_fs.remove(i) # conflict resolved.
                        except:
                            pass
                    break # break the loop since we found the best group
                except:
                    pass
            for conflict_f in conflict_fs: # deal with conflicts
                try: # get w_i's preference for conflict_f
                    w_prefer = P_w[w_i-1].index([conflict_f])
                except:
                    w_prefer = 9e8
                try: # get w_i's preference for current f
                    f_prefered = P_w[w_i-1].index([f+1])
                except:
                    f_prefered = 9e8
                print(w_prefer,f_prefered)
                if w_prefer > f_prefered: # reject proposal, smaller value is prefered
                    print('reject',w_prefer,f_prefered)
                    result_fs.append(conflict_f)
            if len(result_fs) == len(conflict_fs) and len(conflict_fs)>0: 
                mu_f.append(propose_match) # keep current match
                print('mu_f',mu_f,conflict_fs)
                # for w_i in propose_match: # remove the less prefered f from P_w
                #     print(P_w[w_i-1])#[:P_w[w_i-1].index([f+1])])
                #     print(f)
                #     # P_w[w_i-1] = P_w[w_i-1][:P_w[w_i-1].index([f+1])]
                for f_i in result_fs: # resolve other conflict_fs
                    mu_f[f_i-1] = []
                    print('mu_f',mu_f)
                    DA_helper(f_i-1)
            try:
                mu_f[f] = propose_match # case when resolveing other conflict_fs
            except:
                mu_f.append(propose_match) # case when new f is proposing
        print(mu_f)
    while True:
        DA_helper(f)
        f=f+1 # next f
        if f > len(P_f): # done
            break
    return mu_f
        
def DA_result_transpose(P):
    """
        Transpose the representation; switch the index and value.
    """
    n_transposed = max(list(itertools.chain.from_iterable(P)))
    P_t = [[] for i in range(n_transposed)]
    for i,p in enumerate(P):
        for p_i in p:
            print(i,P_t[p_i-1])
            P_t[p_i-1]=P_t[p_i-1]+[i+1]
            print(i,P_t[p_i-1])
    return P_t

                


result_mu_f = DA_algorithm(P_f,P_w)
result_mu_w = DA_result_transpose(DA_algorithm(P_w,P_f))
# index_from_match(2,mu_f)
# DA_result_transpose([[1,2,3]])


ini_prop: [1, 2]
ifm 1 [] []
ifm 2 [[1, 2]] [1]
[[1, 2]]
ini_prop: [3, 6]
ifm 3 [[1, 2]] []
ifm 6 [[1, 2], [3, 6]] [2]
[[1, 2], [3, 6]]
ini_prop: [2, 4]
ifm 2 [[1, 2], [3, 6]] [1]
ifm 4 [[1, 2], [3, 6], [2, 4]] [3]
[[1, 2], [3, 6], [2, 4]]
ini_prop: [3]
ifm 3 [] []
[[3]]
ini_prop: [2, 3]
ifm 2 [[3]] []
ifm 3 [[3], [2, 3]] [1, 2]
[[3], [2, 3]]
ini_prop: [1]
ifm 1 [[3], [2, 3]] []
[[3], [2, 3], [1]]
ini_prop: [1]
ifm 1 [[3], [2, 3], [1]] [3]
[[3], [2, 3], [1], [1]]
ini_prop: [2]
ifm 2 [[3], [2, 3], [1], [1]] [2]
[[3], [2, 3], [1], [1], [2]]
ini_prop: [1, 3]
ini_prop: [3]
ini_prop: [1]
ini_prop: [2]
ifm 2 [[3], [2, 3], [1], [1], [2]] [2, 5]
11 14
[[3], [2, 3], [1], [1], [2], [2]]
0 []
0 [1]
1 []
1 [2]
1 [1]
1 [1, 2]
2 []
2 [3]
3 [3]
3 [3, 4]
4 [2]
4 [2, 5]
5 [2, 5]
5 [2, 5, 6]


In [8]:
print(result_mu_f)
print(result_mu_w)

[[1, 2], [3, 6], [2, 4], []]
[[3, 4], [2, 5, 6], [1, 2]]
